# Read in and Clean Data

In [1]:
# import packages
import os
import pandas as pd
import nltk
from nltk import WhitespaceTokenizer
from nltk.corpus import stopwords
from string import punctuation as original_punct
import re
from collections import Counter
import string
from nltk.stem import WordNetLemmatizer 

# read in data
df = pd.read_csv("Data_IRS_ExcludeMuseumOnly.csv")

# read in list of keywords
keyword_df = pd.read_csv("Keyword_Final_Draft_121521.csv")


### Clean names

In [2]:
# get name column and make lowercase
names = df["name"]
names = [str(name).lower() for name in names]

# remove word equivalents
names2 = []
for name in names:
    name = re.sub(r"\bctr\b", "center", name)
    name = re.sub(r"\bcntr\b", "center", name)
    name = re.sub(r"\bassn\b", "association", name)
    name = re.sub(r"\bassoc\b", " association", name)
    names2.append(name)
    
# create stop word list
nltk.download('stopwords')
stop_words = stopwords.words('english') + list(original_punct)

# define lemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer() 

name_tokens = []
for name in names2:
    name = WhitespaceTokenizer().tokenize(name)           # tokenize
    name = filter(lambda x: x.isalpha(), name)            # remove numerics
    name = [lemmatizer.lemmatize(w) for w in name]        # lemmatize
    name = [w for w in name if not w in stop_words]       # remove stop words 
    name_tokens.append(name)
    
# string tweets together again
names = [" ".join(name) for name in name_tokens] 

df["name"] = names

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ydeng\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ydeng\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Clean keywords

In [11]:
keyword_df = pd.read_csv("Keyword_Final_Draft_121521.csv")

# make keywords lowercase
keywords = keyword_df["Keyword"]
keywords = [str(keyword).lower() for keyword in keywords]


# lemmatize keywords
keyword_tokens = []
for keyword in keywords:
    keyword = WhitespaceTokenizer().tokenize(keyword)           # tokenize
    keyword = [lemmatizer.lemmatize(w) for w in keyword]        # lemmatize
    keyword = [w for w in keyword if not w in stop_words]       # remove stop words 
    keyword_tokens.append(keyword)
    
# string keywords together again and put back in keyword_df
keywords = [" ".join(keyword) for keyword in keyword_tokens]
keyword_df["Keyword"] = keywords

keyword_df.drop_duplicates("Keyword")
keywords = list(set(keywords))

In [12]:
keywords

['aviary',
 'horticultural',
 'hall fame',
 'historial',
 'maritime',
 'consortium',
 'salon',
 'herbarium',
 'birdhouse',
 'game park',
 'game refuge',
 'preservation',
 'pioneer village',
 'adventure childrens',
 'artisan',
 'repository',
 'warbird',
 'conservation',
 'nature reserve',
 'historical',
 'curatorial',
 'botanic',
 'nature center',
 'gallery',
 'historic',
 'learning center',
 'genealogical',
 'national park',
 'atheneum',
 'observatory',
 'art',
 'volary',
 'marine center',
 'library',
 'model',
 'theater',
 'studio',
 'sanctuary',
 'wildlife',
 'legacy',
 'cavern',
 'science center',
 'museo',
 'apparatus',
 'aquarium',
 'technology center',
 'arboretum',
 'theatre',
 'treasury',
 'collector',
 'lighthouse',
 'archaeological',
 'exhibit',
 'conservancy',
 'paleontology',
 'musuem',
 'history',
 'antique',
 'archaeology',
 'marine exhibit',
 'museum',
 'assemblage',
 'battlefield',
 'heritage',
 'nursery',
 'conservatory',
 'dinosaur',
 'zoo',
 'meritime',
 'aboetum',
 

# Output 1: Create Abbreviated Dataframe

In [13]:
# build new abbreviated dataframe with only rows where there is ntee_cd not NA or a keyword match

# df with keyword matches from list 1
is_1 = keyword_df["Include"] == 1
keyword_df_1 = keyword_df[is_1]
df_keyword_1 = pd.DataFrame()
for keyword in keyword_df_1["Keyword"]:
    chunk = df[df['name'].str.contains(keyword)]
    chunk["keyword"] = keyword
    df_keyword_1 = pd.concat([df_keyword_1, chunk], ignore_index=True)  # add rows that contain this keyword

# add "-seum" matches to df_keyword_1
chunk = df[df['name'].str.contains("seum")]  # any containing "museum" will be removed during drop_duplicates later on
chunk["keyword"] = "seum"
df_keyword_1 = pd.concat([df_keyword_1, chunk], ignore_index=True)   
df_keyword_1 = df_keyword_1.drop_duplicates()
    
# df with keyword matches from list 2
is_2 = keyword_df["Include"] == 2
keyword_df_2 = keyword_df[is_2]
df_keyword_2 = pd.DataFrame()
for keyword in keyword_df_2["Keyword"]:
    chunk = df[df['name'].str.contains(keyword)]
    chunk["keyword"] = keyword
    df_keyword_2 = pd.concat([df_keyword_2, chunk], ignore_index=True)  # add rows that contain this keyword
df_keyword_2 = df_keyword_2.drop_duplicates()

# df with non-missing ntee_cd
df_not_na = df[df["ntee_cd"].notna()]
df_not_na["keyword"] = ""

KeyError: 'Include'

In [128]:
# create abbreviated df by combining keyword matches and missing ctee_cds
df_abbr = pd.concat([df_keyword_1, df_keyword_2, df_not_na], ignore_index=True)  # add rows that contain this keyword
df_abbr = df_abbr.drop_duplicates("ein", keep='first')
df_abbr

,ein,name,ico,street,city,state,zip,group,subsection,affiliation,...,sort_name,region,Description,Museum,NonMuseum,Possibly_Museum,no_ntee,unknown_ntee,general_ntee_only,keyword
0,431344660,steel guitar hall fame inc,NaN,9535 MIDLAND BLVD,ST LOUIS,MO,63114-3314,0,3,3,...,NaN,3,NaN,0,0,0,1,0,0,hall fame
1,475114688,philadelphia music hall fame,% GEORGE PETTIGNANO,738 S BROAD ST,PHILADELPHIA,PA,19146-2232,0,3,3,...,NaN,2,Cultural & Ethnic Awareness,0,0,1,0,0,0,hall fame
2,43558201,westford academy athletic hall fame corp,% KEVIN REGAN,PO BOX 926,WESTFORD,MA,01886-0029,0,3,3,...,NaN,1,Youth Development Programs,0,0,1,0,0,0,hall fame
3,473500697,bbe hall fame,NaN,180 BIRCH AVE,BROOTEN,MN,56316-4756,0,3,3,...,NaN,2,Professional Societies & Associations,0,1,0,0,0,0,hall fame
4,611169624,national softball association sport hall fame,NaN,PO BOX 8,NICHOLASVILLE,KY,40340-0008,0,4,3,...,NaN,2,NaN,0,0,0,1,0,0,hall fame
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1541381,450459139,red river radio amateur inc,% LORIANN KRUFT -TREASURE,PO BOX 3215,FARGO,ND,58108-3215,0,3,3,...,NaN,2,Radio,0,1,1,0,0,0,
1541382,463779164,cadena radial mision vision corp,NaN,47250 WASHINGTON ST STE A,LA QUINTA,CA,92253-2105,0,3,3,...,NaN,3,Radio,0,1,1,0,0,0,
1541383,834485399,kisa public radio,% LISA TINSLEY,3379 REGENT PL SW,ATLANTA,GA,30311-3092,0,3,3,...,NaN,3,Radio,0,1,1,0,0,0,
1541384,473452717,inc,% RICHARD J VENEZIA,1494 SW BALMORAL TRAC,STUART,FL,34997-0000,0,3,3,...,NaN,3,Radio,0,1,1,0,0,0,


In [130]:
# save frequencies to csv
df_abbr.to_csv("3_Data_IRS_ExcludeMuseumOnly_abbr.csv")

In [131]:
df_abbr.shape

(1229422, 37)

# Output 2: Count Keyword Frequencies

In [85]:
matching_keywords_1 =df_keyword_1[["keyword", "name", "ntee_cd"]]
    
matching_keywords_1

,keyword,name,ntee_cd
0,hall fame,steel guitar hall fame inc,NaN
1,hall fame,philadelphia music hall fame,A23
2,hall fame,westford academy athletic hall fame corp,O50
3,hall fame,bbe hall fame,N03
4,hall fame,national softball association sport hall fame,NaN
...,...,...,...
34196,seum,brazosport museum natural science,A560
34197,seum,carthage museum historical society,A80
34198,seum,national soccer hall fame museum,N64
34199,seum,amelia island museum history inc,A540


In [86]:
matching_keywords_2 =df_keyword_2[["keyword", "name", "ntee_cd"]]
    
matching_keywords_2

,keyword,name,ntee_cd
0,infantry regiment,virginia volunteer infantry regiment,A82Z
1,infantry regiment,ohio volunteer infantry regiment band,A6C
2,infantry regiment,infantry regiment manchu association,NaN
3,infantry regiment,airborne infantry regiment association,W30
4,infantry regiment,infantry regiment association,P99
...,...,...,...
363704,conserva,coastal bend conservancy,C34
363705,conserva,vigo county conservation club inc,NaN
363706,conserva,hoosier creek conservation retriever association,NaN
363707,conserva,whitefish lake conservation organization inc,C99


In [87]:
matching_keywords_1.to_csv("3_matching_keywords_1.csv")
matching_keywords_2.to_csv("3_matching_keywords_2.csv")

### get frequencies based on keyword

In [88]:
freq_df_1 = matching_keywords_1.groupby(["keyword", "ntee_cd"]).size().reset_index(name='count')
freq_df_2 = matching_keywords_2.groupby(["keyword", "ntee_cd"]).size().reset_index(name='count')
freq_df_1 = freq_df_1.assign(Include=1)
freq_df_2 = freq_df_2.assign(Include=2)
freq_df_1_2 = freq_df_1.append(freq_df_2)

freq_df_2

,keyword,ntee_cd,count,Include
0,ancient,A05,2,2
1,ancient,A20,3,2
2,ancient,A23,16,2
3,ancient,A24Z,1,2
4,ancient,A25,1,2
...,...,...,...,...
13482,zoological,D500,68,2
13483,zoological,D50Z,3,2
13484,zoological,D99,1,2
13485,zoological,O50,1,2


In [89]:
# combine list 1 and 2
freq_df_1_2 = freq_df_1.append(freq_df_2)

In [91]:
# save frequencies to csv
freq_df_1.to_csv("3_keyword_frequencies_1s_only.csv")
freq_df_2.to_csv("3_keyword_frequencies_2s_only.csv")
freq_df_1_2.to_csv("3_keyword_frequencies_1s_2s.csv")

# Output 3: Frequencies Based on ctee_cd

In [101]:
# combine matches from list 1 and 2
matching_keywords = pd.concat([matching_keywords_1, matching_keywords_2], ignore_index=True)
matching_keywords.shape

(397203, 3)

In [120]:
# group by ctee_cd only
freq_df_by_ctee = matching_keywords.groupby(["ntee_cd"]).size().reset_index(name='count').sort_values("count", ascending = False)
freq_df_by_ctee

,ntee_cd,count
2741,T20,19272
303,A80,18638
2769,T22,16388
613,B82,12900
2229,P20,8377
...,...,...
1680,J034,1
1679,J033,1
1675,J022,1
1671,J00I,1


In [123]:
# save freqs by ctee_cd to csv
freq_df_by_ctee.to_csv("3_ctee_frequencies.csv", index = False)